In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True' 

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[ 0.1760,  0.1838, -0.1130, -0.0387, -0.1069,  0.1039,  0.0176,  0.2616,
         -0.0276,  0.0926],
        [ 0.1563,  0.1924, -0.0266, -0.1102, -0.1768, -0.1455, -0.1707,  0.4086,
          0.0296,  0.1239]], grad_fn=<AddmmBackward0>)

In [3]:


class MLP(nn.Module):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    # 定义模型的前向传播，即如何根据输入计算输出
    def forward(self, x):
        x = F.relu(self.hidden(x))
        return self.output(x)

In [4]:
net = MLP()
net(X)

tensor([[-0.0262,  0.1283, -0.0260,  0.0046,  0.1872,  0.1711, -0.1249, -0.0358,
          0.1451,  0.0714],
        [ 0.1067,  0.1435,  0.0745, -0.1416,  0.1852,  0.2668, -0.1359, -0.0833,
         -0.0379,  0.1425]], grad_fn=<AddmmBackward0>)

In [5]:
# 5.1.2 顺序快
class MySequential(nn.Module):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for module in self._modules.values():
            X = module(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0761, -0.0519, -0.1662,  0.0217,  0.0173, -0.0092,  0.1076, -0.0528,
          0.0127, -0.0460],
        [ 0.2270, -0.0425, -0.2158, -0.0144,  0.1583, -0.0474, -0.0488, -0.1352,
         -0.0037,  0.0385]], grad_fn=<AddmmBackward0>)

In [7]:
# 5.1.3 在前向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        x = F.relu(torch.mm(x, self.rand_weight.data) + 1)
        x = self.linear(x)
        while x.abs().sum() > 1:
            x /= 2
        return x.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(0.0248, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, x):
        return self.linear(self.net(x))

In [10]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20),FixedHiddenMLP())
chimera(X)

tensor(-0.1041, grad_fn=<SumBackward0>)